In [28]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split,cross_val_score,RandomizedSearchCV
from sklearn.metrics import r2_score,accuracy_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
import pandas as pd
import numpy as np

In [2]:
data ,y = load_diabetes(return_X_y=True, as_frame=True)

In [4]:
y

0      151.0
1       75.0
2      141.0
3      206.0
4      135.0
       ...  
437    178.0
438    104.0
439    132.0
440    220.0
441     57.0
Name: target, Length: 442, dtype: float64

In [5]:
df= pd.concat([data,y],axis=1)

In [6]:
df

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641,135.0
...,...,...,...,...,...,...,...,...,...,...,...
437,0.041708,0.050680,0.019662,0.059744,-0.005697,-0.002566,-0.028674,-0.002592,0.031193,0.007207,178.0
438,-0.005515,0.050680,-0.015906,-0.067642,0.049341,0.079165,-0.028674,0.034309,-0.018114,0.044485,104.0
439,0.041708,0.050680,-0.015906,0.017293,-0.037344,-0.013840,-0.024993,-0.011080,-0.046883,0.015491,132.0
440,-0.045472,-0.044642,0.039062,0.001215,0.016318,0.015283,-0.028674,0.026560,0.044529,-0.025930,220.0


# 1st step is to calculate the mean of target variable

In [7]:
df['Predict'] = df.target.mean()

# next is to find the residuals

In [9]:
df ['Residuals'] = df.target-df.Predict

In [11]:
df.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target,Predict,Residuals
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646,151.0,152.133484,-1.133484
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204,75.0,152.133484,-77.133484
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930,141.0,152.133484,-11.133484
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362,206.0,152.133484,53.866516
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641,135.0,152.133484,-17.133484


# build a decision tree model by considering Residuals as target 

In [13]:
dt_reg = DecisionTreeRegressor(max_depth=8)

In [16]:
Y_target= df.Residuals
dt_reg.fit(data,Y_target)

DecisionTreeRegressor(max_depth=8)

In [18]:
df['pred_residuals']=dt_reg.predict(data)

In [19]:
df.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target,Predict,Residuals,pred_residuals
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646,151.0,152.133484,-1.133484,65.083907
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204,75.0,152.133484,-77.133484,-57.933484
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930,141.0,152.133484,-11.133484,-18.238747
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362,206.0,152.133484,53.866516,86.866516
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641,135.0,152.133484,-17.133484,-28.919198


# calculate the new predicted output by the formula 
# new predicted output  = Predicted value + (learning_rate * pred_residuals)

In [20]:
learn_rate = 0.1
df['New_Predict'] = df.Predict + learn_rate * df.pred_residuals

In [21]:
df.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target,Predict,Residuals,pred_residuals,New_Predict
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646,151.0,152.133484,-1.133484,65.083907,158.641875
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204,75.0,152.133484,-77.133484,-57.933484,146.340136
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930,141.0,152.133484,-11.133484,-18.238747,150.309609
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362,206.0,152.133484,53.866516,86.866516,160.820136
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641,135.0,152.133484,-17.133484,-28.919198,149.241564


# again follow the step2 from calculation of Residuals

In [22]:
df['Residuals_2']= df.target - df.New_Predict

In [26]:
df[['Residuals','Residuals_2']].head(50)

,Residuals,Residuals_2
0,-1.133484,-7.641875
1,-77.133484,-71.340136
2,-11.133484,-9.309609
3,53.866516,45.179864
4,-17.133484,-14.241564
5,-55.133484,-49.340136
6,-14.133484,-12.720136
7,-89.133484,-80.370136
8,-42.133484,-40.395136
9,157.866516,146.513198


In [27]:
grad_reg = GradientBoostingRegressor(n_estimators=200)

In [32]:
y

0      151.0
1       75.0
2      141.0
3      206.0
4      135.0
       ...  
437    178.0
438    104.0
439    132.0
440    220.0
441     57.0
Name: target, Length: 442, dtype: float64

In [30]:
cross_val_score(grad_reg,data,y,cv=10, scoring='r2',n_jobs=1)

array([0.25431858, 0.17593687, 0.35397808, 0.54183873, 0.17340682,
       0.45572024, 0.36174971, 0.19802638, 0.28292792, 0.40909747])

In [35]:
param = {'n_estimators' : [100,200,300,400,500,700],
         'loss' : ['squared_error', 'absolute_error', 'huber', 'quantile'],
    'learning_rate' : [0.1,0.15,0.2,0.25,0.4,0.5,0.7] ,
    'criterion':['friedman_mse','squared_error'],
    'min_samples_split' :[2,4,5,6,7,10] ,
    'max_depth' : [3,5,7,8,10]
        }

In [36]:
rand_cv = RandomizedSearchCV(grad_reg ,param,cv=10,scoring='r2',n_iter=10,n_jobs=1)

In [37]:
rand_cv

RandomizedSearchCV(cv=10, estimator=GradientBoostingRegressor(n_estimators=200),
                   n_jobs=1,
                   param_distributions={'criterion': ['friedman_mse',
                                                      'squared_error'],
                                        'learning_rate': [0.1, 0.15, 0.2, 0.25,
                                                          0.4, 0.5, 0.7],
                                        'loss': ['squared_error',
                                                 'absolute_error', 'huber',
                                                 'quantile'],
                                        'max_depth': [3, 5, 7, 8, 10],
                                        'min_samples_split': [2, 4, 5, 6, 7,
                                                              10],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500, 700]},
                   scoring='r2')

In [39]:
x_train,x_test,y_train,y_test = train_test_split(data,y,test_size=0.2,random_state=1232)

In [40]:
rand_cv.fit(x_train,y_train)

RandomizedSearchCV(cv=10, estimator=GradientBoostingRegressor(n_estimators=200),
                   n_jobs=1,
                   param_distributions={'criterion': ['friedman_mse',
                                                      'squared_error'],
                                        'learning_rate': [0.1, 0.15, 0.2, 0.25,
                                                          0.4, 0.5, 0.7],
                                        'loss': ['squared_error',
                                                 'absolute_error', 'huber',
                                                 'quantile'],
                                        'max_depth': [3, 5, 7, 8, 10],
                                        'min_samples_split': [2, 4, 5, 6, 7,
                                                              10],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500, 700]},
                   scoring='r2')

In [42]:
rand_cv.best_estimator_

GradientBoostingRegressor(learning_rate=0.15, loss='absolute_error',
                          max_depth=8, min_samples_split=5, n_estimators=400)

In [43]:
rand_cv.best_score_

0.3131519992654352

In [44]:
rand_cv.best_params_

{'n_estimators': 400,
 'min_samples_split': 5,
 'max_depth': 8,
 'loss': 'absolute_error',
 'learning_rate': 0.15,
 'criterion': 'friedman_mse'}